In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import warnings

warnings.filterwarnings('ignore')

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

education = pd.read_sql_query('SELECT * FROM useducation', con=engine)

engine.dispose()

In [3]:
print(education["GRADES_4_G"].count())
print(education["GRADES_8_G"].count())

1361
1361


In [4]:
edu = education.dropna()

### 1) Compute the average score for each row for all students weighted properly by grade. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores.

In [5]:
edu["avg_score"]= ((edu["GRADES_4_G"] * ((edu["AVG_MATH_4_SCORE"] + edu["AVG_READING_4_SCORE"])/2)  + 
                          edu["GRADES_8_G"] * ((edu["AVG_MATH_8_SCORE"] + edu["AVG_READING_8_SCORE"])/2)) /
                         (edu["GRADES_4_G"] + edu["GRADES_8_G"]))

### 2) What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?

In [6]:
score_vs_expenditure = edu[["avg_score", "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE",
              "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]]
score_vs_expenditure.corr()


,avg_score,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
avg_score,1.000000,0.086797,0.100311,0.092225,0.015186,0.001657
TOTAL_EXPENDITURE,0.086797,1.000000,0.989849,0.993285,0.941120,0.928628
INSTRUCTION_EXPENDITURE,0.100311,0.989849,1.000000,0.974872,0.898766,0.885351
SUPPORT_SERVICES_EXPENDITURE,0.092225,0.993285,0.974872,1.000000,0.948014,0.915436
OTHER_EXPENDITURE,0.015186,0.941120,0.898766,0.948014,1.000000,0.921652
CAPITAL_OUTLAY_EXPENDITURE,0.001657,0.928628,0.885351,0.915436,0.921652,1.000000


INSTRUCTION_EXPENDITURE is the most closely correlated with avg_score

### 3) Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

In [9]:
exp = score_vs_expenditure.drop(['TOTAL_EXPENDITURE', 'avg_score'], axis=1)

X = StandardScaler().fit_transform(exp)

sklearn_pca = PCA(n_components= len(exp.columns))
edu["pca_1"] = sklearn_pca.fit_transform(X)

print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    sklearn_pca.explained_variance_ratio_
)

The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [0.94312987 0.03372757 0.01948834 0.00365422]


The percentage total variance for the first component is ~94%

### 4) What is the correlation between the overall score variable and the 1st principal component?

In [11]:
edu[['avg_score', 'pca_1']].corr()

,avg_score,pca_1
avg_score,1.000000,0.054225
pca_1,0.054225,1.000000


The correlation between overall score and 1st principal component is ~.054

### 5) If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

In this particular case, it may not be as wise to use the 1st principal component value since it is not correlated nearly as much to the average score as the expenditure variables. Support services and instruction expenditure are both more closesly correlated and may prove better variables to rely on when fitting a model.